In [2]:
!git clone https://github.com/4paradigm/autox.git
!pip install ./autox

In [3]:
from autox.autox_nlp import NLP_feature
import pandas as pd
import numpy as np
import os
from tqdm import tqdm

In [5]:
df_train = pd.read_csv('sub_train.csv')
df_test = pd.read_csv('sub_val.csv')

In [6]:
use_Toknizer=True
emb_mode = 'Bert'# TFIDF / Word2Vec / Glove / FastText / Bert
encode_mode = 'supervise' # unsupervise / supervise
text_columns_name = ['Title','Synopsis']
target_column = df_train['Price']
candidate_labels=None

In [7]:
nlp = NLP_feature()
nlp.do_mlm = True
nlp.mlm_epochs=3
nlp.model_name = 'microsoft/deberta-v3-base'
nlp.emb_size=100
nlp.n_clusters=20
df = nlp.fit(df_train,
             text_columns_name,
             use_Toknizer,
             emb_mode,
             encode_mode,
             target_column,
             candidate_labels)

Fitting column: Title tokenizer


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Fitting column: Synopsis tokenizer


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Fitting column: Title bert embedding


Some weights of the model checkpoint at microsoft/deberta-v3-base were not used when initializing DebertaV2ForMaskedLM: ['lm_predictions.lm_head.LayerNorm.weight', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.LayerNorm.weight', 'mask_predictions.dense.bias', 'mask_predictions.dense.weight', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.bias', 'deberta.embeddings.position_embeddings.weight', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.LayerNorm.bias']
- This IS expected if you are initializing DebertaV2ForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model

KeyboardInterrupt: 

In [11]:
for column in df.columns:
    df_train[column] = df[column]
df_train = df_train.drop(columns=text_columns_name)

In [12]:
test = nlp.transform(df_test)

loading configuration file ./Title_transformer/config.json
Model config DebertaV2Config {
  "_name_or_path": "./Title_transformer",
  "architectures": [
    "DebertaV2ForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-07,
  "max_position_embeddings": 512,
  "max_relative_positions": -1,
  "model_type": "deberta-v2",
  "norm_rel_ebd": "layer_norm",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "pooler_dropout": 0,
  "pooler_hidden_act": "gelu",
  "pooler_hidden_size": 768,
  "pos_att_type": [
    "p2c",
    "c2p"
  ],
  "position_biased_input": false,
  "position_buckets": 256,
  "relative_attention": true,
  "share_att_key": true,
  "torch_dtype": "float32",
  "transformers_version": "4.16.2",
  "type_vocab_size": 0,
  "vocab_size": 128100
}

loading weights fil

Transforming column: Title


Some weights of the model checkpoint at ./Title_transformer were not used when initializing DebertaV2Model: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of DebertaV2Model were initialized from the model checkpoint at ./Title_transformer.
If your task is similar to the task the model of the checkpoint w

Transforming column: Synopsis


Some weights of the model checkpoint at ./Synopsis_transformer were not used when initializing DebertaV2Model: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing DebertaV2Model from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2Model from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of DebertaV2Model were initialized from the model checkpoint at ./Synopsis_transformer.
If your task is similar to the task the model of the checkp

In [13]:
for column in test.columns:
    df_test[column] = test[column]
df_test = df_test.drop(columns=text_columns_name)

In [15]:
df_train.to_csv(f'{emb_mode}_{encode_mode}_autox_trn.csv',index=False)
df_test.to_csv(f'{emb_mode}_{encode_mode}_autox_val.csv',index=False)

In [ ]:
df_val=pd.read_csv(f'{emb_mode}_{encode_mode}_autox_val.csv').drop(columns=['Price'])
df_val.to_csv(f'{emb_mode}_{encode_mode}_autox_tst.csv',index=False)

In [ ]:
from autox import AutoX

path = f'.' 
autox = AutoX(target = 'Price', train_name = f'{emb_mode}_{encode_mode}_autox_trn.csv', test_name = f'{emb_mode}_{encode_mode}_autox_tst.csv',  id = [], path = path)
sub = autox.get_submit()

In [ ]:
val = pd.read_csv(f'sub_val.csv')
from sklearn.metrics import mean_squared_error
RMSE = np.sqrt(mean_squared_error(val['Price'], sub['Price']))

In [ ]:
RMSE